In [152]:
from source import data_wrangle
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import seaborn as sb
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [153]:
advanced = data_wrangle.read_all_advanced('./data/advanced')
pergame  = data_wrangle.read_all_pergame('./data/per_game')
seasonal = data_wrangle.clean_and_join_seasonal_dataframe(advanced, pergame)
demographic = data_wrangle.read_demographic_data('./data/player_data.csv')
fullstats = data_wrangle.add_years_in_league(seasonal, demographic)

In [154]:
fullstats.groupby('Player').count()['Pos'].max()

20

In [155]:
fullstats.groupby('Player').count()['Pos'].min()

1

In [156]:
onlyfouryears = fullstats[fullstats['Seasons_number'] == 4]

In [157]:
onlyfiveyears = fullstats[fullstats['Seasons_number'] == 5]

In [158]:
single_season_stat_dict = {}
cumulative_season_stat_dict = {}
startyear = 4
endyear = 9 + 1 #needs to be +1 because of range function not being inclusive

#Get single and cumulative stats per year

for i in range(startyear,endyear):
    single_season_stat_dict[i] = fullstats[fullstats['Seasons_number'] == i]
    cumulative_season_stat_dict[i] = fullstats[fullstats['Seasons_number'] <= i]

In [159]:
cumulative_season_stat_dict

{4:                    Player    Pos  Age   Tm   G  MP_total   PER    TS%   3PAr  \
 5           A.J. Bramlett      C   23  CLE   8        61  -0.4  0.190  0.000   
 6             A.J. Guyton     PG   22  CHI  33       630  10.3  0.495  0.359   
 7             A.J. Guyton     PG   23  CHI  45       607  11.8  0.477  0.504   
 8             A.J. Guyton     PG   24  GSW   2         9  -9.0  0.000  0.250   
 9            A.J. Hammons      C   24  DAL  22       163   8.4  0.472  0.238   
 10             A.J. Price     PG   23  IND  56       865  14.0  0.530  0.492   
 11             A.J. Price     PG   24  IND  50       795  10.7  0.454  0.466   
 12             A.J. Price     PG   25  IND  44       568  11.5  0.454  0.506   
 13             A.J. Price     PG   26  WAS  57      1278  12.4  0.501  0.484   
 16           Aaron Brooks     PG   23  HOU  51       608  13.1  0.535  0.484   
 17           Aaron Brooks     PG   24  HOU  80      1998  12.9  0.521  0.395   
 18           Aaron Brook

First pass attempt, let's just use year 4 to predict year 5

In [160]:
statstokeep = ['WS',
 'VORP',
 'MP_total',
 'PS/G',
 'FG',
 'MP_pergame',
 'GS',
 'FT',
 '2P',
 'FTA',
 'FGA',
 '2PA',
 'DRB', 'Player']

In [161]:
rf = RandomForestRegressor()

In [162]:
to_predict = onlyfiveyears[['Player','WS']]

In [163]:
reduced = onlyfouryears[statstokeep]

In [164]:
players = reduced.pop('Player')

In [165]:
year4players = players

In [166]:
year5players = set(onlyfiveyears['Player'])

In [171]:
year4players = set(year4players)

So we have a bunch of guys that are not in because they started in 2015 (Nerlens, Dante Exum, etc.--the 2014 draft class)

In [172]:
for player in year4players:
    if player not in year5players:
        print(player, "is not in year 5")

Tarik Black is not in year 5
Donte Greene is not in year 5
Joe Ingles is not in year 5
Joe Harris is not in year 5
Dino Radja is not in year 5
Hamady N'Diaye is not in year 5
Luke Jackson is not in year 5
Manny Harris is not in year 5
Bruno Caboclo is not in year 5
Maurice Taylor is not in year 5
Glenn Robinson (ii) is not in year 5
Shabazz Napier is not in year 5
Hollis Thompson is not in year 5
Adreian Payne is not in year 5
Gary Harris is not in year 5
Zach LaVine is not in year 5
Mario West is not in year 5
Jared Cunningham is not in year 5
Rudy Fernandez is not in year 5
Paul Davis is not in year 5
Rashad McCants is not in year 5
Tierre Brown is not in year 5
Elfrid Payton is not in year 5
Todd MacCulloch is not in year 5
Kendall Marshall is not in year 5
Pablo Prigioni is not in year 5
Clint Capela is not in year 5
DaJuan Summers is not in year 5
Dexter Pittman is not in year 5
Lonny Baxter is not in year 5
Yakhouba Diawara is not in year 5
James Anderson is not in year 5
Anthony

In [174]:
player = 'Nerlens Noel'
demographic.loc[demographic['name']==player,'year_start']==2015

2958    True
Name: year_start, dtype: bool

In [175]:
demographic.loc[(demographic['name']==player) & (demographic['year_start']==2015)]

,name,year_start,year_end,position,height,weight,birth_date,college
2958,Nerlens Noel,2015,2018,C-F,6-11,228.0,"April 10, 1994",University of Kentucky


In [176]:
(demographic[demographic['name']==player]['year_start'] == 2015).bool()

True

In [177]:
rows_to_add = []

for player in year4players:
    if player not in year5players and not (demographic[demographic['name']==player]['year_start'] == 2015).bool():
        print(player, "is not in year 5 and didn't start in 2015; started in: ", int(demographic[demographic['name']==player]['year_start']))
        rows_to_add.append([player, 0])
        

Donte Greene is not in year 5 and didn't start in 2015; started in:  2009
Dino Radja is not in year 5 and didn't start in 2015; started in:  1994
Hamady N'Diaye is not in year 5 and didn't start in 2015; started in:  2011
Luke Jackson is not in year 5 and didn't start in 2015; started in:  2005
Manny Harris is not in year 5 and didn't start in 2015; started in:  2011
Maurice Taylor is not in year 5 and didn't start in 2015; started in:  1998
Hollis Thompson is not in year 5 and didn't start in 2015; started in:  2014
Mario West is not in year 5 and didn't start in 2015; started in:  2008
Jared Cunningham is not in year 5 and didn't start in 2015; started in:  2013
Rudy Fernandez is not in year 5 and didn't start in 2015; started in:  2009
Paul Davis is not in year 5 and didn't start in 2015; started in:  2007
Rashad McCants is not in year 5 and didn't start in 2015; started in:  2006
Tierre Brown is not in year 5 and didn't start in 2015; started in:  2002
Todd MacCulloch is not in yea

So Derrick Rose is in this group but didn't play year 5 because of his ACL tear.  There are probably some others. Like Seth Curry (NOT STEPH).

Who is in year 5 that isn't in year 4?

In [183]:
year5players - year4players

{'Adam Keefe',
 'Alonzo Mourning',
 'Andrew Goudelock',
 'Anthony Peeler',
 'Blake Ahearn',
 'Brent Price',
 'Bryant Stith',
 'Charles Smith (ii)',
 'Charlie Bell',
 'Chris Douglas-Roberts',
 'Christian Laettner',
 'Clarence Weatherspoon',
 'Corey Benjamin',
 'Dajuan Wagner',
 'Darrell Arthur',
 'Darryl Watkins',
 'Derrick Dial',
 'Don MacLean',
 'Donny Marshall',
 'Doug Christie',
 'Doug Overton',
 'Dwight Buycks',
 'Earl Barron',
 'Elmore Spencer',
 'Eric Riley',
 'Gerald Madkins',
 'Harold Ellis',
 'Hassan Whiteside',
 'Hubert Davis',
 'Jamie Watson',
 'Jim Jackson',
 'Joe Courtney',
 'John Amaechi',
 'John Crotty',
 'Jon Barry',
 'Junior Harrington',
 'Justin Harper',
 'Kaniel Dickens',
 'LaPhonso Ellis',
 'Latrell Sprewell',
 'Lee Mayberry',
 'Litterial Green',
 'Lloyd Daniels',
 'Lorenzo Brown',
 'Lorenzo Williams',
 'Maceo Baston',
 'Malik Sealy',
 'Matt Geiger',
 'Maurice Ager',
 'Michael Ruffin',
 'Morris Almond',
 'Nate Wolters',
 'Oliver Miller',
 'Othyus Jeffers',
 'Quincy 

Why are younger guys, like Shane Larkin, Mo Ager, Hassan Whiteside, and Darrell Arthur not in year 4?

Whiteside didn't play in his 3rd and 4th year.
Larkin didn't play in his 4th.
Ager played in Europe.
Arthur tore his ACL.


So who from year 4 isn't in the year-4 data who didn't enter the league in 1993?


In [186]:
for player in year5players:
    if player not in year4players and not (demographic[demographic['name']==player]['year_start'] == 1993).bool():
        print(player, "is not in year 4 and didn't start in 1993; started in: ", int(demographic[demographic['name']==player]['year_start']))
       

Chris Douglas-Roberts is not in year 4 and didn't start in 1993; started in:  2009
Earl Barron is not in year 4 and didn't start in 1993; started in:  2006
Blake Ahearn is not in year 4 and didn't start in 1993; started in:  2008
Derrick Dial is not in year 4 and didn't start in 1993; started in:  2000
Roger Mason is not in year 4 and didn't start in 1993; started in:  2003
Nate Wolters is not in year 4 and didn't start in 1993; started in:  2014
Shane Larkin is not in year 4 and didn't start in 1993; started in:  2014
Darrell Arthur is not in year 4 and didn't start in 1993; started in:  2009
Rodney Buford is not in year 4 and didn't start in 1993; started in:  2000
Eric Riley is not in year 4 and didn't start in 1993; started in:  1994
Shawnelle Scott is not in year 4 and didn't start in 1993; started in:  1997
Rusty LaRue is not in year 4 and didn't start in 1993; started in:  1998
Lorenzo Brown is not in year 4 and didn't start in 1993; started in:  2014
Shawne Williams is not in y

In [182]:
year4players

{'A.J. Price',
 'Aaron Brooks',
 'Aaron Gordon',
 'Aaron Gray',
 'Aaron McKie',
 'Aaron Williams',
 'Acie Earl',
 'Acie Law',
 'Adam Morrison',
 'Adonal Foyle',
 'Adreian Payne',
 'Adrian Griffin',
 'Al Harrington',
 'Al Horford',
 'Al Jefferson',
 'Al Thornton',
 'Al-Farouq Aminu',
 'Alan Henderson',
 'Alec Burks',
 'Alex Acker',
 'Alex Len',
 'Allan Houston',
 'Allen Crabbe',
 'Allen Iverson',
 'Alonzo Gee',
 'Alvin Williams',
 "Amar'e Stoudemire",
 'Amir Johnson',
 'Anderson Varejao',
 'Andray Blatche',
 'Andre Barrett',
 'Andre Drummond',
 'Andre Iguodala',
 'Andre Miller',
 'Andre Roberson',
 'Andrea Bargnani',
 'Andrei Kirilenko',
 'Andres Nocioni',
 'Andrew Bogut',
 'Andrew Bynum',
 'Andrew DeClercq',
 'Andrew Nicholson',
 'Andrew Wiggins',
 'Andris Biedrins',
 'Anfernee Hardaway',
 'Ansu Sesay',
 'Antawn Jamison',
 'Anthony Bennett',
 'Anthony Carter',
 'Anthony Davis',
 'Anthony Johnson',
 'Anthony Miller',
 'Anthony Morrow',
 'Anthony Randolph',
 'Anthony Roberson',
 'Anthony

In [126]:
to_predict_dataframe = to_predict.append(pd.DataFrame(rows_to_add, columns = ['Player', 'WS'])).set_index('Player')

In [127]:
to_predict_dataframe

,WS
Player,
A.J. Price,0.0
Aaron Gray,1.4
Aaron McKie,1.6
Aaron Williams,1.7
Adam Keefe,2.3
Adonal Foyle,1.2
Adrian Griffin,0.1
Al Harrington,3.9
Al Horford,1.3


In [200]:
onlyfouryears = onlyfouryears[statstokeep]

In [202]:
onlyfouryears.head()

,WS,VORP,MP_total,PS/G,FG,MP_pergame,GS,FT,2P,FTA,FGA,2PA,DRB
Player,,,,,,,,,,,,,
A.J. Price,2.2,0.1,1278,7.7,2.8,22.4,22,0.9,1.6,1.1,7.2,3.7,1.6
Aaron Brooks,1.1,-0.5,1284,10.7,3.7,21.8,12,2.1,2.5,2.4,9.9,5.9,1.0
Aaron Gordon,2.9,1.0,1909,17.6,6.5,32.9,57,2.7,4.5,3.9,14.9,9.0,6.4
Aaron Gray,1.1,0.0,531,3.1,1.4,13.0,6,0.4,1.4,0.8,2.4,2.4,2.7
Aaron McKie,1.4,0.3,1813,4.1,1.7,22.4,32,0.5,1.6,0.7,4.7,3.9,2.1


In [201]:
onlyfouryears = onlyfouryears.set_index('Player')

In [204]:
onlyfouryears.head()

,WS,VORP,MP_total,PS/G,FG,MP_pergame,GS,FT,2P,FTA,FGA,2PA,DRB
Player,,,,,,,,,,,,,
A.J. Price,2.2,0.1,1278,7.7,2.8,22.4,22,0.9,1.6,1.1,7.2,3.7,1.6
Aaron Brooks,1.1,-0.5,1284,10.7,3.7,21.8,12,2.1,2.5,2.4,9.9,5.9,1.0
Aaron Gordon,2.9,1.0,1909,17.6,6.5,32.9,57,2.7,4.5,3.9,14.9,9.0,6.4
Aaron Gray,1.1,0.0,531,3.1,1.4,13.0,6,0.4,1.4,0.8,2.4,2.4,2.7
Aaron McKie,1.4,0.3,1813,4.1,1.7,22.4,32,0.5,1.6,0.7,4.7,3.9,2.1


In [205]:
onlyfouryears = onlyfouryears.sort_index()

In [206]:
year4fromdf = set(onlyfouryears.index.values)

In [207]:
len(onlyfouryears)

883

In [208]:
len(to_predict_dataframe)

911

In [209]:
year4players = set(year4players)



In [210]:
len(year4players - year5players)

174

In [211]:
inthepredict = set(to_predict_dataframe.index.values)

In [212]:
len(inthepredict - year4fromdf)

78

In [213]:
guystoremovefromyear4df = year4fromdf-inthepredict

In [214]:
guystoremovefrompredict = inthepredict - year4fromdf

In [215]:
onlyfouryears.head()

,WS,VORP,MP_total,PS/G,FG,MP_pergame,GS,FT,2P,FTA,FGA,2PA,DRB
Player,,,,,,,,,,,,,
A.J. Price,2.2,0.1,1278,7.7,2.8,22.4,22,0.9,1.6,1.1,7.2,3.7,1.6
Aaron Brooks,1.1,-0.5,1284,10.7,3.7,21.8,12,2.1,2.5,2.4,9.9,5.9,1.0
Aaron Gordon,2.9,1.0,1909,17.6,6.5,32.9,57,2.7,4.5,3.9,14.9,9.0,6.4
Aaron Gray,1.1,0.0,531,3.1,1.4,13.0,6,0.4,1.4,0.8,2.4,2.4,2.7
Aaron McKie,1.4,0.3,1813,4.1,1.7,22.4,32,0.5,1.6,0.7,4.7,3.9,2.1


In [219]:
for player in guystoremovefromyear4df:
    onlyfouryears.drop(player,inplace=True)

In [224]:
len(onlyfouryears)

833

In [230]:
for player in guystoremovefrompredict:
    to_predict_dataframe.drop(player,inplace=True)
    

In [231]:
len(to_predict_dataframe)

833

In [232]:
trainidx = set(onlyfouryears.index.values)

In [233]:
predictidx = set(to_predict_dataframe.index.values)

In [234]:
trainidx - predictidx

set()

In [235]:
trainidx^predictidx

set()

In [236]:
onlyfouryears.index.equals(to_predict_dataframe.index)

False

In [237]:
predictidx-trainidx

set()

In [239]:
to_predict_dataframe = to_predict_dataframe.sort_index()

In [240]:
onlyfouryears.index.equals(to_predict_dataframe.index)

True

In [242]:
rf = RandomForestRegressor(n_jobs=-1,oob_score=True)

rf.fit(onlyfouryears,to_predict_dataframe)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:732: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [243]:
rf.oob_score_

0.32782833906028164

In [ ]:
rf